### Chargement des modules nécessaires pour le projet
Commencez par exécuter la cellule ci-dessous. Elle va charger tout le nécessaire pour faire fonctionner votre code. 
Si vous le lancez en ligne, __soyez patients__ (attendez que l'étoile à gauche devienne un nombre entre crochets).


In [1]:
include("jupy.jl")

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]
  Resolving package versions...
   Updating `~/.julia/environments/v1.4/Project.toml`
 [no changes]
   Updating `~/.julia/environments/v1.4/Manifest.toml`
 [no changes]


main (generic function with 1 method)

### Donnéees à utiliser

On rappelle les structures qui sont remplies par les méthodes de lecture de données.

#### data structure for data used to compute the balance cost
```
struct BalanceData
    resource1::Int64        # first resource
    resource2::Int64        # second resource
    target::Int64           # target aimed for the recource consumption
    weight::Int64           # weight of the balance cost
end
```

#### data struture for an instance of Google challenge problem
```
struct DataGoogle
    nbResources::Int64               # number of resources in the instance
    transientStatus::Array{Int64}    # table indicating if each resource is transient (O/1)
    weightLoadCost::Array{Int64}     # table indicating the weight of load cost for each resource

    nbMachines::Int64                # number of machines
    neighborhoods::Array{Int64}      # table indicating the neighborhood of each machine
    nbNeighborhoods::Int64           # total number of neighbordhoods
    locations::Array{Int64}          # table indicating the location of each machine
    nbLocations::Int64               # total number of locations
    softCapacities::Array{Int64,2}   # softcapacities[m,r] indicates the soft cap of machine m for resource r
    hardCapacities::Array{Int64,2}   # hardcapacities[m,r] indicates the hard cap of machine m for resource r
    machineMoveCosts::Array{Int64,2} # machineMoveCosts[m1,m2] indicates the cost for moving from m1 to m2

    nbServices::Int64                # number of services
    spreadMin::Array{Int64}          # for each service, its spread min
    dependences::Array{Array{Int64}} # for each service, a list of dependences (service numbers)

    nbProcess::Int64                 # number of processes
    servicesProcess::Array{Int64}    # for each process  its service
    processReq::Array{Int64,2}       # processReq[p,r] indicates the consumption of resource r by p
    processMoveCost::Array{Int64}    # cost of moving each process p

    nbBalanceCostData::Int64               # number of balance constraints
    balanceCostDataList::Array{BalanceData} # one for each balance target
    processMoveWeight::Int64        # weight for the process move cost
    serviceMoveWeight::Int64        # weight for the service move cost
    machineMoveWeight::Int64        # weight for the machine move cost

    initialAssignment # for each process, id of the machine in the initial solution
end
```

#### slution 
```
struct SolutionGoogle
    assignment::Array{Int64}
    cost::Int64
end
```





### Cellule à modifier
Insérez ci-dessous votre modèle dans la méthode **solveGoogle**. 

N'oubliez pas de faire un run sur la cellule quand vous avez fini. 

In [22]:
using Cbc
using JuMP
function solveGoogle(data::DataGoogle, verbose::Bool)
   
    #################################################################################
    ######### Ce code a été réalisé par Mohamed Amine Amghar et Imad Boudroua #######
    #################################################################################
    
     model = Model(Cbc.Optimizer)
    
    set_optimizer_attribute(model, "seconds", 60)
    
    ############################
    ######### Question1  #######
    ############################

    # xij des booléens = 1 si le processus i affecté à la machine j, 0 sinon
    
    
    @variable(model,x[i=1:data.nbProcess,j=1:data.nbMachines],Bin)
    

    ########################################### C1 #####################################################
    #pour tout processus i sum(xij for j in [|1,nbr_machines|]) == 1 
    
    @constraint(model, constraint_1[i=1:data.nbProcess], sum(x[i,j] for j=1:data.nbMachines)  == 1)
    
    ########################################### C2 #####################################################

    #pour toute machine m et ressourse r sum(R(i,r)*xim <= C(m,r) for i in [|1,nbr_processus|])

    @constraint(model, constraint_2[m=1:data.nbMachines, r=1:data.nbResources], sum(data.processReq[i,r]*x[i,m] 
                for i=1:data.nbProcess) <= data.hardCapacities[m,r])
    
    
    ########################################### C3 #####################################################
    #si deux processus fp et sp différent et appartiennent aux meme service donc qlq machine x[fp,machine] + x[sp,machine] <=1
    #c-a-d que x[fp,machine] et x[sp,machine] ne peuvent pas Être tout les deux = 1
    
    for p1 in 1:data.nbProcess
        for p2 in 1:data.nbProcess
            if (p1 != p2) && (data.servicesProcess[p1] == data.servicesProcess[p2])
                    @constraint(model, [j in 1:data.nbMachines], x[p1,j] + x[p2,j] <= 1)
            end
        end
    end

    
    ########################################### O3 #####################################################
    #si processus change de machine initiale alors x[processus,m0(processus)] = 0 => on compte data.processMoveCost[processus]
    #si processus ne change pas de machine initiale alors x[processus,m0(processus)] = 1 => on compte 0

    @variable(model, O3)
    
    @constraint(model, O3 == sum((1 - x[p, data.initialAssignment[p]]) * data.processMoveCost[p] 
                for p in 1:data.nbProcess))
    
    
    
    ############################
    ######### Question2  #######
    ############################


    ########################################### O1 #####################################################
    #sum(data.weightLoadCost[r] * max(U-Sc,0))
    #on calcule le max(a,b) par :
    #           max >= a
    #           max >= b
    # (max <= a + M * y et max <= b + M *(1 - y)) : pas la peine puisqu'on va faire min dans @objective,
    #ce qui vérifie cette contrainte et minimise le temps d'execution  
    
    @variable(model, O1, Int)
    
    @variable(model, U[1:data.nbMachines,1:data.nbResources],Int)
    
    @variable(model, Max[1:data.nbMachines,1:data.nbResources],Int)
    
    for w in 1:data.nbMachines
        for f in 1:data.nbResources
            @constraint(model, U[w,f] == sum(x[p,w] * data.processReq[p,f] for p in 1:data.nbProcess))
            @constraint(model, Max[w,f] >= U[w,f] - data.softCapacities[w,f])
            @constraint(model, Max[w,f] >= 0)
        end
    end
    
    @constraint(model, O1 == sum(Max[m,r]*data.weightLoadCost[r]
            for m in 1:data.nbMachines, r in 1:data.nbResources))
    
    

    ########################################### O2 #####################################################
    #sum(max(0,e12 * A1 - A2))
    #on calcule le max par la meme methode de O1

    @variable(model, O2, Int)
    
    @variable(model, A[1:data.nbMachines,1:data.nbResources],Int)
    
    @variable(model, Mx[1:data.nbMachines,1:data.nbBalanceCostData],Int)
    
    for w in 1:data.nbMachines
        for f in 1:data.nbResources
            @constraint(model, A[w,f] == data.hardCapacities[w,f] - U[w,f])
        end
        for i in 1:data.nbBalanceCostData
            @constraint(model, Mx[w,i] >= data.balanceCostDataList[i].target *
                        A[w,data.balanceCostDataList[i].resource1] - A[w,data.balanceCostDataList[i].resource2])
            @constraint(model, Mx[w,i] >= 0)
        end
    end
    
    @constraint(model, O2 == sum(Mx[w,i]*data.balanceCostDataList[i].weight 
                for w in 1:data.nbMachines, i in 1:data.nbBalanceCostData))
    
    
    
    ########################################### O4 #####################################################

    
    @variable(model, O4, Int)
    
    for s in 1:data.nbServices
        @constraint(model, O4 >= sum(1 - x[process, data.initialAssignment[process]] 
                    for process in 1:data.nbProcess if s==data.servicesProcess[process]))
    end
    
    ########################################### O5 #####################################################
    #sum(sum(move_cost[initialAssignment[p],i]*x[p,i] for p in [|1,nbr_processus|] for i in [|1,nbr_machines|]))

    
     @variable(model , O5)
    
     @constraint(model , O5 == sum(sum(data.machineMoveCosts[data.initialAssignment[process],i]*x[process,i] 
                for process in 1:data.nbProcess) for i in 1:data.nbMachines))
    
    
    
    
    ############################
    ######### Question3  #######
    ############################



    ########################################### C4 #####################################################

    
    
    
    @variable(model, srvLocation[1:data.nbServices,1:data.nbLocations])
    
    @constraint(model, constraint_4[s=1:data.nbServices,i=1:data.nbLocations], srvLocation[s,i] -
                sum(x[p,m] for p=1:data.nbProcess, m=1:data.nbMachines 
                if data.servicesProcess[p] == s && data.locations[m] == i) == 0)
    
    @constraint(model, C4[s=1:data.nbServices], sum(1 for i=1:data.nbLocations if srvLocation[s,i] != 0) 
                >= data.spreadMin[s])
    
    
    
    @objective(model, Min, O1 + O2 + data.processMoveWeight * O3 + data.serviceMoveWeight * O4 + 
                data.machineMoveWeight * O5)
    
    
    ############################
    ######### Question4  #######
    ############################
        
    ########################################### C5 #####################################################
    
    """
    # the number of machines used by a service in a neighborhood
    @variable(mod, service_in_neighborhood[1:data.nbServices,1:data.nbNeighborhoods] >= 0)
    @constraint(mod,nbMachinesNeighborhood[s=1:data.nbServices,n=1:data.nbNeighborhoods],service_in_neighborhood[s,n] - sum(x[p,m]
            for m in 1:data.nbMachines,
                p in 1:data.nbProcess
                if
                    data.servicesProcess[p] == s && data.neighborhoods[m] == n) == 0)
    
    #The number of services that want n to be a neighborhood of service s
    @variable(mod,neighborhoodService[1:data.nbServices,1:data.nbNeighborhoods] >= 0)
    
    @constraint(mod,nbServicesNeighborhood[s=1:data.nbServices,n=1:data.nbNeighborhoods],neighborhoodService[s,n] - sum(1
            for s0 in data.dependences[s]
                if
                    service_in_neighborhood[s0,n] >= 1) == 0)
    
    #Each mandatory neighborhood of service s should be used by service s
    @constraint(mod,C5[s=1:data.nbServices],sum(1
            for n in 1:data.nbNeighborhoods
                if
                   neighborhoodService[s,n] != 0 &&  service_in_neighborhood[s,n] == 0) == 0)
    """
    ############################
    ######### Question5  #######
    ############################

    ########################################### C6 #####################################################
    for i in 1:data.nbMachines
        for r in 1:data.nbResources
            if data.transientStatus[r]==1
                @constraint(model, sum(data.initialAssignment[j]+x[j,i] * data.processReq[j,r] for j=1:data.nbProcess) <= data.hardCapacities[i,r])
            end
        end
    end
    
    set_optimizer_attribute(model, "logLevel", 0)
    optimize!(model) 
    
    objval = JuMP.objective_value(model)
    soluce = Array{Int64}(undef, data.nbProcess)
    
    for i in 1:data.nbProcess
        soluce[i] = sum(k*round(JuMP.value(x[i, k])) for k=1:data.nbMachines)
    end
    return SolutionGoogle(soluce, objval)
    
end

function solveGoogle(data::DataGoogle, verbose::Bool)
   
    #################################################################################
    ######### Ce code a été réalisé par Mohamed Amine Amghar et Imad Boudroua #######
    #################################################################################
    
     model = Model(Cbc.Optimizer)
    
    set_optimizer_attribute(model, "seconds", 60)
    
    ############################
    ######### Question1  #######
    ############################

    # xij des booléens = 1 si le processus i affecté à la machine j, 0 sinon
    
    
    @variable(model,x[i=1:data.nbProcess,j=1:data.nbMachines],Bin)
    

    ########################################### C1 #####################################################
    #pour tout processus i sum(xij for j in [|1,nbr_machines|]) == 1 
    
    @constraint(model, constraint_1[i=1:data.nbProcess], sum(x[i,j] for j=1:data.nbMachines)  == 1)
    
    ########################################### C2 #####################################################

    #pour toute machine m et ressourse r sum(R(i,r)*xim <= C(m,r) for i in [|1,nbr_processus|])

    @constraint(model, constraint_2[m=1:data.nbMachines, r=1:data.nbResources], sum(data.processReq[i,r]*x[i,m] 
                for i=1:data.nbProcess) <= data.hardCapacities[m,r])
    
    
    ########################################### C3 #####################################################
    #si deux processus fp et sp différent et appartiennent aux meme service donc qlq machine x[fp,machine] + x[sp,machine] <=1
    #c-a-d que x[fp,machine] et x[sp,machine] ne peuvent pas Être tout les deux = 1
    
    for p1 in 1:data.nbProcess
        for p2 in 1:data.nbProcess
            if (p1 != p2) && (data.servicesProcess[p1] == data.servicesProcess[p2])
                    @constraint(model, [j in 1:data.nbMachines], x[p1,j] + x[p2,j] <= 1)
            end
        end
    end

    
    ########################################### O3 #####################################################
    #si processus change de machine initiale alors x[processus,m0(processus)] = 0 => on compte data.processMoveCost[processus]
    #si processus ne change pas de machine initiale alors x[processus,m0(processus)] = 1 => on compte 0

    @variable(model, O3)
    
    @constraint(model, O3 == sum((1 - x[p, data.initialAssignment[p]]) * data.processMoveCost[p] 
                for p in 1:data.nbProcess))
    
    
    
    ############################
    ######### Question2  #######
    ############################


    ########################################### O1 #####################################################
    #sum(data.weightLoadCost[r] * max(U-Sc,0))
    #on calcule le max(a,b) par :
    #           max >= a
    #           max >= b
    # (max <= a + M * y et max <= b + M *(1 - y)) : pas la peine puisqu'on va faire min dans @objective,
    #ce qui vérifie cette contrainte et minimise le temps d'execution  
    
    @variable(model, O1, Int)
    
    @variable(model, U[1:data.nbMachines,1:data.nbResources],Int)
    
    @variable(model, Max[1:data.nbMachines,1:data.nbResources],Int)
    
    for w in 1:data.nbMachines
        for f in 1:data.nbResources
            @constraint(model, U[w,f] == sum(x[p,w] * data.processReq[p,f] for p in 1:data.nbProcess))
            @constraint(model, Max[w,f] >= U[w,f] - data.softCapacities[w,f])
            @constraint(model, Max[w,f] >= 0)
        end
    end
    
    @constraint(model, O1 == sum(Max[m,r]*data.weightLoadCost[r]
            for m in 1:data.nbMachines, r in 1:data.nbResources))
    
    

    ########################################### O2 #####################################################
    #sum(max(0,e12 * A1 - A2))
    #on calcule le max par la meme methode de O1

    @variable(model, O2, Int)
    
    @variable(model, A[1:data.nbMachines,1:data.nbResources],Int)
    
    @variable(model, Mx[1:data.nbMachines,1:data.nbBalanceCostData],Int)
    
    for w in 1:data.nbMachines
        for f in 1:data.nbResources
            @constraint(model, A[w,f] == data.hardCapacities[w,f] - U[w,f])
        end
        for i in 1:data.nbBalanceCostData
            @constraint(model, Mx[w,i] >= data.balanceCostDataList[i].target *
                        A[w,data.balanceCostDataList[i].resource1] - A[w,data.balanceCostDataList[i].resource2])
            @constraint(model, Mx[w,i] >= 0)
        end
    end
    
    @constraint(model, O2 == sum(Mx[w,i]*data.balanceCostDataList[i].weight 
                for w in 1:data.nbMachines, i in 1:data.nbBalanceCostData))
    
    
    
    ########################################### O4 #####################################################

    
    @variable(model, O4, Int)
    
    for s in 1:data.nbServices
        @constraint(model, O4 >= sum(1 - x[process, data.initialAssignment[process]] 
                    for process in 1:data.nbProcess if s==data.servicesProcess[process]))
    end
    
    ########################################### O5 #####################################################
    #sum(sum(move_cost[initialAssignment[p],i]*x[p,i] for p in [|1,nbr_processus|] for i in [|1,nbr_machines|]))

    
     @variable(model , O5)
    
     @constraint(model , O5 == sum(sum(data.machineMoveCosts[data.initialAssignment[process],i]*x[process,i] 
                for process in 1:data.nbProcess) for i in 1:data.nbMachines))
    
    
    
    
    ############################
    ######### Question3  #######
    ############################



    ########################################### C4 #####################################################

    
    
    
    @variable(model, srvLocation[1:data.nbServices,1:data.nbLocations])
    
    @constraint(model, constraint_4[s=1:data.nbServices,i=1:data.nbLocations], srvLocation[s,i] -
                sum(x[p,m] for p=1:data.nbProcess, m=1:data.nbMachines 
                if data.servicesProcess[p] == s && data.locations[m] == i) == 0)
    
    @constraint(model, C4[s=1:data.nbServices], sum(1 for i=1:data.nbLocations if srvLocation[s,i] != 0) 
                >= data.spreadMin[s])
    
    
    
    @objective(model, Min, O1 + O2 + data.processMoveWeight * O3 + data.serviceMoveWeight * O4 + 
                data.machineMoveWeight * O5)
    
    
    ############################
    ######### Question4  #######
    ############################
        
    ########################################### C5 #####################################################
    #Cette question n'a pas été testé à cause d'un temps d'éxecution très grand (soucis matériel)
    
    """
    # the number of machines used by a service in a neighborhood
    @variable(mod, service_in_neighborhood[1:data.nbServices,1:data.nbNeighborhoods] >= 0)
    @constraint(mod,nbMachinesNeighborhood[s=1:data.nbServices,n=1:data.nbNeighborhoods],service_in_neighborhood[s,n] - sum(x[p,m]
            for m in 1:data.nbMachines,
                p in 1:data.nbProcess
                if
                    data.servicesProcess[p] == s && data.neighborhoods[m] == n) == 0)
    
    #The number of services that want n to be a neighborhood of service s
    @variable(mod,neighborhoodService[1:data.nbServices,1:data.nbNeighborhoods] >= 0)
    
    @constraint(mod,nbServicesNeighborhood[s=1:data.nbServices,n=1:data.nbNeighborhoods],neighborhoodService[s,n] - sum(1
            for s0 in data.dependences[s]
                if
                    service_in_neighborhood[s0,n] >= 1) == 0)
    
    #Each mandatory neighborhood of service s should be used by service s
    @constraint(mod,C5[s=1:data.nbServices],sum(1
            for n in 1:data.nbNeighborhoods
                if
                   neighborhoodService[s,n] != 0 &&  service_in_neighborhood[s,n] == 0) == 0)
    """
    ############################
    ######### Question5  #######
    ############################

    ########################################### C6 #####################################################
    for i in 1:data.nbMachines
        for r in 1:data.nbResources
            if data.transientStatus[r]==1
                @constraint(model, sum(data.initialAssignment[j]+x[j,i] * data.processReq[j,r] for j=1:data.nbProcess) <= data.hardCapacities[i,r])
            end
        end
    end
    
    set_optimizer_attribute(model, "logLevel", 0)
    optimize!(model) 
    
    objval = JuMP.objective_value(model)
    soluce = Array{Int64}(undef, data.nbProcess)
    
    for i in 1:data.nbProcess
        soluce[i] = sum(k*round(JuMP.value(x[i, k])) for k=1:data.nbMachines)
    end
    return SolutionGoogle(soluce, objval)
    
end

function solveGoogle(data::DataGoogle, verbose::Bool)
   
    #################################################################################
    ######### Ce code a été réalisé par Mohamed Amine Amghar et Imad Boudroua #######
    #################################################################################
    
     model = Model(Cbc.Optimizer)
    
    set_optimizer_attribute(model, "seconds", 60)
    
    ############################
    ######### Question1  #######
    ############################

    # xij des booléens = 1 si le processus i affecté à la machine j, 0 sinon
    
    
    @variable(model,x[i=1:data.nbProcess,j=1:data.nbMachines],Bin)
    

    ########################################### C1 #####################################################
    #pour tout processus i sum(xij for j in [|1,nbr_machines|]) == 1 
    
    @constraint(model, constraint_1[i=1:data.nbProcess], sum(x[i,j] for j=1:data.nbMachines)  == 1)
    
    ########################################### C2 #####################################################

    #pour toute machine m et ressourse r sum(R(i,r)*xim <= C(m,r) for i in [|1,nbr_processus|])

    @constraint(model, constraint_2[m=1:data.nbMachines, r=1:data.nbResources], sum(data.processReq[i,r]*x[i,m] 
                for i=1:data.nbProcess) <= data.hardCapacities[m,r])
    
    
    ########################################### C3 #####################################################
    #si deux processus fp et sp différent et appartiennent aux meme service donc qlq machine x[fp,machine] + x[sp,machine] <=1
    #c-a-d que x[fp,machine] et x[sp,machine] ne peuvent pas Être tout les deux = 1
    
    for p1 in 1:data.nbProcess
        for p2 in 1:data.nbProcess
            if (p1 != p2) && (data.servicesProcess[p1] == data.servicesProcess[p2])
                    @constraint(model, [j in 1:data.nbMachines], x[p1,j] + x[p2,j] <= 1)
            end
        end
    end

    
    ########################################### O3 #####################################################
    #si processus change de machine initiale alors x[processus,m0(processus)] = 0 => on compte data.processMoveCost[processus]
    #si processus ne change pas de machine initiale alors x[processus,m0(processus)] = 1 => on compte 0

    @variable(model, O3)
    
    @constraint(model, O3 == sum((1 - x[p, data.initialAssignment[p]]) * data.processMoveCost[p] 
                for p in 1:data.nbProcess))
    
    
    
    ############################
    ######### Question2  #######
    ############################


    ########################################### O1 #####################################################
    #sum(data.weightLoadCost[r] * max(U-Sc,0))
    #on calcule le max(a,b) par :
    #           max >= a
    #           max >= b
    # (max <= a + M * y et max <= b + M *(1 - y)) : pas la peine puisqu'on va faire min dans @objective,
    #ce qui vérifie cette contrainte et minimise le temps d'execution  
    
    @variable(model, O1, Int)
    
    @variable(model, U[1:data.nbMachines,1:data.nbResources],Int)
    
    @variable(model, Max[1:data.nbMachines,1:data.nbResources],Int)
    
    for w in 1:data.nbMachines
        for f in 1:data.nbResources
            @constraint(model, U[w,f] == sum(x[p,w] * data.processReq[p,f] for p in 1:data.nbProcess))
            @constraint(model, Max[w,f] >= U[w,f] - data.softCapacities[w,f])
            @constraint(model, Max[w,f] >= 0)
        end
    end
    
    @constraint(model, O1 == sum(Max[m,r]*data.weightLoadCost[r]
            for m in 1:data.nbMachines, r in 1:data.nbResources))
    
    

    ########################################### O2 #####################################################
    #sum(max(0,e12 * A1 - A2))
    #on calcule le max par la meme methode de O1

    @variable(model, O2, Int)
    
    @variable(model, A[1:data.nbMachines,1:data.nbResources],Int)
    
    @variable(model, Mx[1:data.nbMachines,1:data.nbBalanceCostData],Int)
    
    for w in 1:data.nbMachines
        for f in 1:data.nbResources
            @constraint(model, A[w,f] == data.hardCapacities[w,f] - U[w,f])
        end
        for i in 1:data.nbBalanceCostData
            @constraint(model, Mx[w,i] >= data.balanceCostDataList[i].target *
                        A[w,data.balanceCostDataList[i].resource1] - A[w,data.balanceCostDataList[i].resource2])
            @constraint(model, Mx[w,i] >= 0)
        end
    end
    
    @constraint(model, O2 == sum(Mx[w,i]*data.balanceCostDataList[i].weight 
                for w in 1:data.nbMachines, i in 1:data.nbBalanceCostData))
    
    
    
    ########################################### O4 #####################################################

    
    @variable(model, O4, Int)
    
    for s in 1:data.nbServices
        @constraint(model, O4 >= sum(1 - x[process, data.initialAssignment[process]] 
                    for process in 1:data.nbProcess if s==data.servicesProcess[process]))
    end
    
    ########################################### O5 #####################################################
    #sum(sum(move_cost[initialAssignment[p],i]*x[p,i] for p in [|1,nbr_processus|] for i in [|1,nbr_machines|]))

    
     @variable(model , O5)
    
     @constraint(model , O5 == sum(sum(data.machineMoveCosts[data.initialAssignment[process],i]*x[process,i] 
                for process in 1:data.nbProcess) for i in 1:data.nbMachines))
    
    
    
    
    ############################
    ######### Question3  #######
    ############################



    ########################################### C4 #####################################################

    
    
    
    @variable(model, srvLocation[1:data.nbServices,1:data.nbLocations])
    
    @constraint(model, constraint_4[s=1:data.nbServices,i=1:data.nbLocations], srvLocation[s,i] -
                sum(x[p,m] for p=1:data.nbProcess, m=1:data.nbMachines 
                if data.servicesProcess[p] == s && data.locations[m] == i) == 0)
    
    @constraint(model, C4[s=1:data.nbServices], sum(1 for i=1:data.nbLocations if srvLocation[s,i] != 0) 
                >= data.spreadMin[s])
    
    
    
    @objective(model, Min, O1 + O2 + data.processMoveWeight * O3 + data.serviceMoveWeight * O4 + 
                data.machineMoveWeight * O5)
    
    
    ############################
    ######### Question4  #######
    ############################
        
    ########################################### C5 #####################################################
    #Cette question n'a pas été testé à cause d'un temps d'éxecution très grand (soucis matériel)
    
    """
    # the number of machines used by a service in a neighborhood
    @variable(mod, service_in_neighborhood[1:data.nbServices,1:data.nbNeighborhoods] >= 0)
    @constraint(mod,nbMachinesNeighborhood[s=1:data.nbServices,n=1:data.nbNeighborhoods],service_in_neighborhood[s,n] - sum(x[p,m]
            for m in 1:data.nbMachines,
                p in 1:data.nbProcess
                if
                    data.servicesProcess[p] == s && data.neighborhoods[m] == n) == 0)
    
    #The number of services that want n to be a neighborhood of service s
    @variable(mod,neighborhoodService[1:data.nbServices,1:data.nbNeighborhoods] >= 0)
    
    @constraint(mod,nbServicesNeighborhood[s=1:data.nbServices,n=1:data.nbNeighborhoods],neighborhoodService[s,n] - sum(1
            for s0 in data.dependences[s]
                if
                    service_in_neighborhood[s0,n] >= 1) == 0)
    
    #Each mandatory neighborhood of service s should be used by service s
    @constraint(mod,C5[s=1:data.nbServices],sum(1
            for n in 1:data.nbNeighborhoods
                if
                   neighborhoodService[s,n] != 0 &&  service_in_neighborhood[s,n] == 0) == 0)
    """
    ############################
    ######### Question5  #######
    ############################

    ########################################### C6 #####################################################
    for i in 1:data.nbMachines
        for r in 1:data.nbResources
            if data.transientStatus[r]==1
                @constraint(model, sum(data.initialAssignment[j]+x[j,i] * data.processReq[j,r] for j=1:data.nbProcess) <= data.hardCapacities[i,r])
            end
        end
    end
    
    set_optimizer_attribute(model, "logLevel", 0)
    optimize!(model) 
    
    objval = JuMP.objective_value(model)
    soluce = Array{Int64}(undef, data.nbProcess)
    
    for i in 1:data.nbProcess
        soluce[i] = sum(k*round(JuMP.value(x[i, k])) for k=1:data.nbMachines)
    end
    return SolutionGoogle(soluce, objval)
    
end



LoadError: syntax: incomplete: "for" at none:93 requires end

### Cellule de test
La cellule ci-dessous permet d'exécuter le code. Elle lit la donnée, lance votre méthode solveGoogle, récupère la solution, et vérifie le respect des contraintes et son coût. 

Vous pouvez modifier le nom pour tester sur un autre jeu de donnée. 

In [21]:
# enter your instance name here 
donnee = "a1_1"  # possibles values a1_1, a1_2, a1_3, a1_4, a1_5, a2_1, a2_2, a2_3, a2_4, a2_5 

# set to yes if you want to print intermediate logs 
verbose = false

main("model_"*donnee*".txt","assignment_"*donnee*".txt",verbose)


Received instance files model_a1_1.txt and assignment_a1_1.txt
---------Nbre machines----------
4
---------Nbre processus---------
100
--------------------------------
4.4306501e7
Checking the solution 
Solution received is [1, 1, 1, 2, 2, 4, 4, 4, 4, 3, 2, 1, 1, 4, 4, 2, 4, 1, 1, 4, 3, 2, 2, 2, 1, 2, 2, 3, 1, 4, 4, 4, 1, 3, 3, 1, 1, 2, 1, 4, 1, 1, 2, 4, 2, 4, 1, 3, 3, 3, 1, 4, 1, 4, 4, 1, 2, 2, 3, 1, 1, 3, 3, 2, 1, 3, 4, 2, 2, 1, 1, 3, 1, 1, 4, 1, 2, 1, 2, 4, 4, 2, 1, 1, 1, 4, 1, 1, 1, 3, 1, 4, 4, 2, 3, 4, 3, 2, 2, 3]
Cost : 44306501
Checking the size of the vector
Test passed
Checking that the values in the solution are machine indices
Test passed
Checking capacity constraint (without transient)
Test passed
Checking capacity constraint (with transient)
Test passed
Checking disjunctions between processes of the same service
Test passed
Checking spread constraints 
Test passed
Checking dependency constraints
Test passed
All tests passed: solution feasible 
Load cost = 3.101173e7
Balanc